In [27]:
import numpy as np
from matplotlib import pyplot as plt
from prova import *






In [30]:
sorgenti = ["americio", "sodio1", "cesio", "cobalto1", "sodio2", "cobalto2"]
file = [r"Data\290323\290323_americio2.dat",
        r"Data\290323\290323_sodio2.dat",
        r"Data\290323\290323_cesio2.dat",
        r"Data\290323\290323_cobalto2.dat",
        r"Data\290323\290323_sodio2.dat",
        r"Data\290323\290323_cobalto2.dat"]
inf = np.array([286, 2565, 3331, 5820, 6133, 6565])
sup = np.array([400, 2840, 3800, 6280, 6862, 7020])
theta = np.radians(22)

media = []
err_media = []

larghezza = []
err_larghezza = []

for i in range(6):
     popt, pcov =best_fit(file[i], inf[i], sup[i], sorgenti[i])





_______________________________
Media americio :  317.9109328072518
Errore media americio :  0.27465121258571856
_______________________________
Sigma americio :  21.63698931423562
Errore sigma americio :  0.21735010518064182
Ampiezza:  16031.217273332966
Chisquare = 1582.6
63.303308655565736
_______________________________
Media sodio1 :  2697.9768893954733
Errore media sodio1 :  1.5357832056037133
_______________________________
Sigma sodio1 :  98.40141460442791
Errore sigma sodio1 :  2.125028060051069
Ampiezza:  1416.0309438813745
Chisquare = 30.9
2.210616329771279
_______________________________
Media cesio :  3500.303853496632
Errore media cesio :  1.1889707286029856
_______________________________
Sigma cesio :  109.38742412838955
Errore sigma cesio :  1.0025229514812815
Ampiezza:  4876.75978059082
Chisquare = 144.6
5.560800927509131
_______________________________
Media cobalto1 :  6011.280457453264
Errore media cobalto1 :  2.3745247791768938
_______________________________
Sigm

TypeError: expected x and y to have same length

In [29]:
print(media)

[]
